##  rc2rot  RC and q calculation (2 direction (psi) data) 

Method A(RC2rot): RC -> trim -> rotation 

psi = 0, 120 (-120) or 0, 90 (-90)

(1) Check the angle file

(2) Rocking curve (RC) 

- HW fitting angle = 0, 120

- gauss fitting angle = 0, 120

Note: The fitting time depends on the capability of your PC.

(3) Image trimming

(4) Prepare the rotating data using image_editor

(5) Calculate q

(6) re-plot and re-analysis 


In [ ]:
# from pathlib import Path

# import matplotlib.pyplot as plt
# import mpl_toolkits.axes_grid1
# from mpl_toolkits.mplot3d import Axes3D

# import numpy as np
# import pandas as pd
# from PIL import Image

In [ ]:
from qfit import make_angle_file as maf
from qfit import fit_q
from qfit import re_analysis as rean
from qfit import re_plot as replt
from qfit import multiplot as mlplt
from qfit import image_treat as imt
from qfit import file_folder_trans as fft

# Detector size
NX=2368
NY=2240

In [ ]:
import warnings
warnings.simplefilter('ignore')
# warnings.resetwarnings()
# warnings.simplefilter('ignore', FutureWarning)
%load_ext autoreload
%autoreload 2

In [ ]:
# Set target folder path
# g_0 = r'C:\Users\USER\Desktop\XRTD\2_inch_0'
# g_120 = r'C:\Users\USER\Desktop\XRTD\2_inch_120'

g_0 = r""
g_120 = r""

### Check angle file

In [ ]:
# Check if it contains tif data, angle.text, and dark data.
# psi=0
_ = maf.check_holder(data_path=g_0)

# psi=120
_ = maf.check_holder(data_path=g_120)

# If the angle file does not exist, go back to creating the angle file. -> temp_anglefile.ipynb

In [ ]:
# If the angle file exists, check the angle file
maf.anglefile_info(data_path=g_0)
print('-'*5)
maf.anglefile_info(data_path=g_120)

### RC calculation

In [ ]:
# RC hw fitting

# core: Use multi-process PC cores. Attention memory size.

# filter: Filter parameters are set to exclude noise level signals from the analysis. 
# It is only analyzed if the difference between the maximum and minimum RC intensities is greater than the filter value. 
# If it is not included in the analysis, enter the value of Nan as the analysis value.

# method: hw (full width half maxima) 

cores = 6
t_file = g_0
t_folder,_ = fit_q.fit_analysis(target_file=t_file, filter=30, method='hw', comment='0data', core=cores)
print('')

c_file = g_120
c_folder,_ = fit_q.fit_analysis(target_file=c_file, filter=30, method='hw', comment='120data', core=cores)
print('')


# RC gauss fitting

# pmax: In addition to the filter condition, 
# if the condition with low distribution intensity (maximum intensity> median intensity + PMAX) is not satisfied,
# it is excluded from the analysis. If it is not analyzed, or if the fitting fails, the value of Nan is entered.

# method: gaussian 

# t_file = g_0
# t_folder = fit_q.fit_analysis(target_file=t_file, filter=30, pmax=30, method='gaussian', comment='0data', core=cores)
# print('')

# c_file = g_120
# c_folder = fit_q.fit_analysis(target_file=c_file, filter=30, pmax=30, method='gaussian', comment='120data', core=cores)
# print('')


In [ ]:
# Check results
rc0 = rean.load_rc_tif(t_folder)
mlplt.rc_12plots(rc0, title='$\psi$=0')

rc120 = rean.load_rc_tif(c_folder)
mlplt.rc_12plots(rc120, title='$\psi$=120')

### Trimming

<img src="./docs/figs/triming1.JPG" style="zoom:25%;">
<img src="./docs/figs/triming2.JPG" style="zoom:25%;">

- Wait a few seconds and a new GUI window will appear.
- Area select: Right-click 
- Decide and quit: Press 'ESC' key

In [ ]:
# psi=0
# 2 inch: wh(width,height)=1100, 4 inch: wh=2150, M :wh=500
#  Wait a few seconds and a new GUI window will appear.
wh4 = 2150
wh2 = 1100
whm = 500
wh_set = wh2
tr_t_folder = imt.gui2trim(t_folder, wh=wh_set, NX=NX, NY=NY, time_out=120)

In [ ]:
# psi=120
tr_c_folder=imt.gui2trim(c_folder, wh=wh_set, NX=NX, NY=NY, time_out=120)

In [ ]:
# Check results
rc0 = rean.load_rc_tif(tr_t_folder)
mlplt.rc_12plots(rc0, title='$\psi$=0')

rc120 = rean.load_rc_tif(tr_c_folder)
mlplt.rc_12plots(rc120, title='$\psi$=120')

### Rotating image using Image_editor

<img src="./docs/figs_affin/img_edit.PNG" style="zoom:40%;">

(1) Upper left window: Select data folder (psi=0)

(2) Set the reading step (1: all files, default 16)

(3) Middle left window: Select data folder (e.g. psi=120)

(4) Set the reading step (1: All files, default 16)

(5) If you need to flip the x-direction, set the scale original x to -1.

(6) Change the x,y, rotation, and scale parameters.

(7) Export from the File menu.

(8) Quit from the File menu.



In [ ]:
#  Wait a few seconds and a new GUI window will appear.
outfolderlist,_ = imt.img_editor_process()
print(outfolderlist)

# out put folder name :rot_ + 'load folder name' ex: rot_tr_hw_220208_132247
# if missing outfolderlist, input the folder name.
# outfolderlist = ['','']

In [ ]:
# Check results
rc0 = rean.load_rc_tif(outfolderlist[0])
mlplt.rc_12plots(rc0, title='$\psi$=0')

rc120 = rean.load_rc_tif(outfolderlist[1])
mlplt.rc_12plots(rc120, title='$\psi$=120')

### h Calculation (Q calculation)

In [ ]:
# Set the data folder after rotation.

# fr0=r'C:\Users\USER\Desktop\XRTD\xrdt\tr_hw_211226_215327'
# fr120=r'C:\Users\USER\Desktop\XRTD\xrdt\tr_hw_211226_215523'

fr0 = outfolderlist[0]
fr120 = outfolderlist[1]

print(fr0,fr120)

In [ ]:
# convert tif to npy

r0 = fft.conv_tif2npy(file_dir=fr0)
r120 = fft.conv_tif2npy(file_dir=fr120)

# fft.fd_tif2fd_npy(file_dir=fr0)
# fft.fd_tif2fd_npy(file_dir=fr120)

# r0=fft.folder_file_list(fr0,look_for='npy')
# r120=fft.folder_file_list(fr120,look_for='npy')

print('psi=0')
print(r0)

print('psi=120')
print(r120)

In [ ]:
# The '_c.npy' file is used to calculate q.

target_t_ = r0[0]
target_c_ = r120[0]

# set output folder name
out_file_ = '2R_0120'
# phi=0
set_angle_t = 0 
# phi=120
set_angle_c = 120

# If it is trimmed, change NX, NY.
# original NX=2368, NY=2240, 
out_q_folder = fit_q.q_analysis_2R(target_t=target_t_, target_c=target_c_, out_file=out_file_, 
                                    angle_t=set_angle_t, angle_c=set_angle_c, 
                                    q=6.258 ,NX=wh_set, NY=wh_set)

In [ ]:
# Check results
q_0120 = rean.load_q_tif(out_q_folder)
mlplt.qcp_12plots(q_0120,title='q (0,120)',step=50)
mlplt.qp3_4plots(q_0120, title='q polar',step=50)
mlplt.qp3_3plots(q_0120, title='q polar analysis',step=50)